<a href="https://colab.research.google.com/github/markiroberts/gogglecolab01/blob/main/03_pt01_Scrape_website_create_matches_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [181]:
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup

years = list(range(2023,2020, -1))
all_matches = []
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
for year in years:
  pl_stats = requests.get(standings_url)
  pl_stats_soup = BeautifulSoup(pl_stats.text)
  standings_table_soup = pl_stats_soup.select('table.stats_table')[0]
  standings_table_links = standings_table_soup.find_all('a')

  squad_links_set = set()
  for link in standings_table_links:
    m = link.get("href")
    if '/squads' in m:
      m = f"https://fbref.com{m}"
      squad_links_set.add(m)

  previous_season_href = pl_stats_soup.select("a.prev")[0].get("href")
  standings_url = f"https://fbref.com{previous_season_href}"

  for team_url in squad_links_set:
    team_name = team_url.split("/")[-1].replace("-Stats","").replace("-"," ")
    print(f"year {year} | team {team_name}")
    team_data = requests.get(team_url)
    matches = pd.read_html(team_data.text, match="Scores & Fixtures")[0]
    matches = matches.drop(columns=['Notes','Match Report'])
    matches = matches.dropna()
    team_soup = BeautifulSoup(team_data.text)
    team_links = team_soup.find_all('a')
    team_shoot_links = set()
    for link in team_links:
        m = link.get("href")
        if m:
          if 'all_comps/shooting' in m:
            m = f"https://fbref.com{m}"
            team_shoot_links.add(m)
    shooting_data = requests.get(list(team_shoot_links)[0])
    team_shooting = pd.read_html(shooting_data.text, match="Shooting")[0]
    team_shooting.columns = team_shooting.columns.droplevel()
    try:
      team_data = matches.merge(team_shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]],on="Date")
    except ValueError:
      continue
    team_data = team_data[team_data["Comp"] == "Premier League"]
    team_data["Season"] = year
    team_data["Team"] = team_name
    all_matches.append(team_data)
    time.sleep(1)

match_df = pd.concat(all_matches)
match_df.columns = [c.lower() for c in match_df.columns]
match_df.to_csv("matches.csv")

year 2023 | team Nottingham Forest
year 2023 | team Aston Villa
year 2023 | team Newcastle United
year 2023 | team West Ham United
year 2023 | team Bournemouth
year 2023 | team Everton
year 2023 | team Crystal Palace
year 2023 | team Manchester United
year 2023 | team Arsenal
year 2023 | team Liverpool
year 2023 | team Fulham
year 2023 | team Burnley
year 2023 | team Brentford
year 2023 | team Brighton and Hove Albion
year 2023 | team Tottenham Hotspur
year 2023 | team Manchester City
year 2023 | team Sheffield United
year 2023 | team Chelsea
year 2023 | team Luton Town
year 2023 | team Wolverhampton Wanderers
year 2022 | team West Ham United
year 2022 | team Leicester City
year 2022 | team Tottenham Hotspur
year 2022 | team Liverpool
year 2022 | team Crystal Palace
year 2022 | team Manchester United
year 2022 | team Everton
year 2022 | team Leeds United
year 2022 | team Aston Villa
year 2022 | team Brentford
year 2022 | team Arsenal
year 2022 | team Newcastle United
year 2022 | team W